In [1]:
!pip install emoji
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/carlospenaloza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from textblob import TextBlob
from bs4 import BeautifulSoup
import matplotlib.cm as cm
from string import punctuation
from string import digits
import urllib.request
import networkx as nx
import pandas as pd
import itertools
import emoji
import math
import time
import json
import sys
import re

# TODO: remove this when NetworkX is fixed
from warnings import simplefilter
import matplotlib.cbook
simplefilter("ignore", matplotlib.cbook.mplDeprecation)

In [24]:
# Import tweets
tweets = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/tweets/statuses.csv')

tweets.head()

,id,tweet_id,created_at,text,favorite_count,retweet_count,phone,sensitive,hashtags,no_hashtags,mentions,no_mentions,no_urls,no_media
0,113127283.0,1.172520e+18,9/13/19 14:56,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
1,113127283.0,1.172520e+18,9/13/19 14:54,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,2,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",5,0,1
2,113127283.0,1.172520e+18,9/13/19 14:52,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
3,113127283.0,1.172520e+18,9/13/19 14:50,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
4,113127283.0,1.172520e+18,9/13/19 14:48,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",4,0,1


In [7]:
# Import spanish stop word dictionary
url_sw = 'https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/stopwords-es.json'
response_sw = urllib.request.urlopen(url_sw)
data_sw = response_sw.read()

stop_words = set(json.loads(data_sw))

# Import emoji meanings
emoji_translations = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/emojis_translated.csv')

# Get spanish meaning of an emoji
def emoji_meaning(emoji):
    meaning = emoji_translations.loc[emoji_translations['emoji'] == emoji]['translation']
    return meaning.values[0] if not meaning.empty else ''

# Transformations to remove digits and punctuation
remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans('', '', punctuation)

In [8]:
from ipython_exit import exit

""" Processes text data in tweets
    
    text: text of tweet
    word_counts: word count dictionary
    
    returns processed text, length of content, polarity, subjectivity
"""
def process_text(text, word_counts, user_id):
    # Remove links
    text = ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())
    
    # Remove file names
    
        
    # Remove mentions
    text = ' '.join(re.sub("(@[A-Za-z0-9^\w]+)", " ", text.replace('@ ','@').replace('# ','#')).split())
        
    # Replace hashtags with words
    if text.count('#')>0:
        text = ' '.join(re.findall('[A-Z][^A-Z]*', text.replace('#', ' ')))
            
    #Remove HTML tags
    text = BeautifulSoup(text).get_text()
    
    # Save content length (exluding links and mentions)
    length = len(text)
        
    # Remove punctuation symbols
    text = ' '.join(re.sub("[\.\,\¡\¿\!\?\:\;\-\=\*\(\)\[\]\"\'\“\_\+\”\%\/\‘\’]", " ", text).split())
    text = text.translate(remove_digits).translate(remove_punctuation)
        
    # Lower case to avoid case sensitive problems
    text = text.lower()
        
    # Replace emojis with names 
    text = emoji.demojize(text)
        
    # Add space between emojis and other characters
    ind = -2
    for c in range(text.count(':')):
        ind = text.find(':',ind+2)
        if c%2==0:
            newLetter = ' :'
        else:
            newLetter = ': '
        text ="".join((text[:ind],newLetter,text[ind+1:]))
            
    # Replace emoji names with spanish meaning
    result = []
    parts = text.split(' ')
    for part in parts:
        if part:
            if part[0]==':':
                em = handle_emoji_tone(part)
                em = emoji_meaning(em)
                if em:
                    result.append(em)
            else:
                if part not in stop_words:
                    if part not in word_counts:
                        word_counts[part] = 1
                    else:
                        word_counts[part] += 1
                result.append(part)
        
    text = ' '.join(result)
        
    # Filter using NLTK library append it to a string
    word_tokens = word_tokenize(text)
        
    result = [w for w in word_tokens if not w in stop_words]
        
    text = ' '.join(result)
    
    if len(text)>12:
        analysis = TextBlob(text)
        try:
            eng = analysis.translate(to='en')
            sentiment = eng.sentiment     
            polarity = sentiment.polarity
            subjectivity = sentiment.subjectivity
            time.sleep(.55)
        except Exception as e:
            if str(e)=='HTTP Error 429: Too Many Requests':
                print('Error')
                print('Last user:', user_id)
                exit()
            polarity = 0.0
            subjectivity = 0.0
    
    else:
        polarity = 0.0
        subjectivity = 0.0
    

    return text, length, polarity, subjectivity

tones = ['_light_skin_tone','_medium-light_skin_tone','_medium_skin_tone', 
             '_medium-dark_skin_tone','_dark_skin_tone']

# Method that removes the tone from emojis
def handle_emoji_tone(emoji):  
    for t in tones:
        if t in emoji:
            tone = t
            return emoji.replace(tone,'')
    return emoji


In [9]:
test = '@Tolaymaruja Las esperamos en el cine este fin de semana de estreno de @monosfilm #MONOS las abraza. https://t.co/L5B7xs9M3e '
process_text(test,{},1)

('esperamos cine semana estreno abraza', 81, 0.0, 0.0)

In [10]:
# Import the network
G = nx.read_graphml("network1.graphml")

# List to hold part of the dataset
text_analysis = []
text_analysis.append(['id', 'tweet_id','engagement',
            'effective_length', 'polarity', 'subjectivity'])

# List to store average engagement
avg_eng = []
avg_eng.append(['id', 'average_engagement', 'common_words', 'common_ht_words',
               'common_hts'])

In [11]:
''' Text analysis '''

last_uid = 3284817049
found_start = False

#for n in G.nodes():
for n in list(G.nodes()):
    user_id = int(float(n))
    #if not found_start and user_id != last_uid:
    #    continue
    #else:
    #    found_start = True
    
    print ('User: ', user_id)
    
    # Total engagement
    eng = 0
    tweet_count = 0
    
    # Frequent word dictionary
    freq_words = {}

    # Frequent hashtag dictionary
    freq_hashtags = {}

    # Frequent words in hashtags dictionary
    freq_ht_words = {}
    
    user_tweets = tweets.loc[tweets['id'] == user_id]
    for index, row in user_tweets.iterrows():
      
        # Add current engagement
        currEng = int(row['favorite_count'])+int(row['retweet_count'])
        eng += currEng
        tweet_count +=1
        
        # Get hashtags
        ht = row['hashtags'].split(';') if row['hashtags'] != '-' else []
        
        # Update hashtag count
        for hashtag in ht:
            if hashtag not in freq_hashtags:
                freq_hashtags[hashtag] = 1
            else:
                freq_hashtags[hashtag] += 1
        
        # Separate hashtags by capitalization
        ht_words = [re.findall('[a-zA-Z][^A-Z]*', w) for w in ht]
        
        # Create a list with all the words in the hashtags
        hts = []
        for h in ht_words:
            hts = hts + h
        
        # Remove digits and lower caps for every hashtag word
        hts = [item.translate(remove_digits).lower() for item in hts]
        
        # Update hashtag word frequencies
        for word in hts:
            if word not in stop_words:
                if word not in freq_ht_words:
                    freq_ht_words[word] = 1
                else:
                    freq_ht_words[word] += 1
        
        text, length, polarity, subjectivity = process_text(row['text'], freq_words, user_id)
        
        # Update text in  data frame
        text_analysis.append([n, row['tweet_id'], currEng, length, polarity, subjectivity])
        print('Original:',row['text'],'\nProcessed:',text,'\nPolarity, subjectivity:',
              polarity, subjectivity,'\n')
    
    if tweet_count > 0:
        average_engagement = eng / tweet_count
    else:
        average_engagement = 0
    c_words = Counter(freq_words)
    c_ht_words = Counter(freq_ht_words)
    c_hashtags = Counter(freq_hashtags)
    avg_eng.append([user_id, average_engagement, c_words.most_common(3),
                   c_ht_words.most_common(3), c_hashtags.most_common(3)])

User:  426146744
Original: Partido de la selección, cervecita, amigos, y de remate Pipe Bueno y Paola Jara ¿nos vemos allá? #EfectyElGiroOficial https://t.co/AWVIjpwXLb 
Processed: partido selección cervecita amigos remate pipe paola jara vemos allá efecty giro oficial 
Polarity, subjectivity: 0.6 0.8 

Original: Hay gente descarada en la vida y este par. Qué gonorrea parce. https://t.co/WndKKPV8sH 
Processed: gente descarada vida par gonorrea parce 
Polarity, subjectivity: 0.0 0.0 

Original: No olvidemos JAJAJAA https://t.co/aOQHnIqA03 
Processed: olvidemos jajajaa 
Polarity, subjectivity: 0.2 0.4 

Original: Paaarce, me encanta como @CervezaAguila premia a los verdaderos #Creyentes de la selección. Esto lo motiva más a uno para seguir apoyando a la tricolor en cada partido ¡VAMOS COLOMBIA! https://t.co/LdgP4i0PQL 
Processed: paaarce encanta premia verdaderos creyentes selección motiva seguir apoyando tricolor partido 
Polarity, subjectivity: 0.3666666666666667 0.5 

Original: Si van

Original: Santa Marta es magia🤩 Un destino lleno de vida e historia 💛✈️ https://t.co/F6qeJePRDo 
Processed: santa marta magia destino lleno vida historia corazon amarillo avión 
Polarity, subjectivity: 0.2833333333333333 0.5166666666666667 

Original: @hmsebastiann ¡Nos la puso de pa´rriba! 😊 en https://t.co/q0OjyVHvZg! Los precios están de locura ¡Corre que se agotan!  💨 ✈️ 💛 
Processed: puso pa´rriba precios locura corre agotan avión corazon amarillo 
Polarity, subjectivity: 0.0 0.0 

Original: @EffArturo ¡Hola! En Viva Air trabajamos constantemente por el cuidado del medio ambiente 🌳🌎 y ante la petición realizada en el portal https://t.co/8UxsFVTn48 ¡tenemos una gran noticia! finalizando este año contaremos con la posibilidad de generar el pasabordo digital. ¡Feliz tarde! 💛 
Processed: hola viva air constantemente cuidado ambiente árbol hoja caduca petición realizada portal noticia finalizando año contaremos posibilidad generar pasabordo digital feliz corazon amarillo 
Polarity, sub

Original: @alexanderarcila @ChangeorgCO ¡Hola! En Viva Air trabajamos constantemente por el cuidado del medio ambiente 🌳🌎 y ante la petición realizada en el portal https://t.co/8UxsFVTn48 ¡tenemos una gran noticia! finalizando este año contaremos con la posibilidad de generar el pasabordo digital. ¡Linda tarde! 💛 
Processed: hola viva air constantemente cuidado ambiente árbol hoja caduca petición realizada portal noticia finalizando año contaremos posibilidad generar pasabordo digital linda corazon amarillo 
Polarity, subjectivity: 0.125 0.3333333333333333 

Original: @mggarciab @wingo @ChangeorgCO ¡Hola! En Viva Air trabajamos constantemente por el cuidado del medio ambiente 🌳🌎 y ante la petición realizada en el portal https://t.co/8UxsFVTn48 ¡tenemos una gran noticia! finalizando este año contaremos con la posibilidad de generar el pasabordo digital. ¡Linda tarde! 💛 
Processed: hola viva air constantemente cuidado ambiente árbol hoja caduca petición realizada portal noticia finalizan

Original: 🏆⚽️Se dio a conocer la lista de árbitros que dirigirán el partidos correspondientes a la fecha 15 de la #LigaAguila II- 2019 y fecha 14 del #TorneoAguila II-2019.⚽️🏆  👉https://t.co/bvMkEIHrO0 https://t.co/SgadzZu0p3 
Processed: lista árbitros dirigirán partidos correspondientes fecha liga aguila fecha torneo aguila balón fútbol trofeo índice revés apuntando derecha 
Polarity, subjectivity: 0.19285714285714284 0.46785714285714286 

Original: 🏆⚽📊Aquí algunos datos previos a la fecha 15 de la #LigaAguila II-2019📊⚽🏆  👉https://t.co/rAs0wuBCza https://t.co/BjKxUNUUDS 
Processed: datos previos fecha liga aguila gráfico barras balón fútbol trofeo índice revés apuntando derecha 
Polarity, subjectivity: 0.03968253968253968 0.3674603174603175 

Original: #LigaAguila 🏆   💥⚽️Disputada la fecha 14, así quedó la tabla de posiciones de la #LigaAguila II-2019💥⚽️ https://t.co/oWdST6KXoc 
Processed: liga aguila trofeo colisión balón fútbol disputada fecha tabla posiciones liga aguila colisión b

Original: 📆⚽️Nos permitimos informar la programación de la fecha 17 en la #LigaAguila II-2019.⚽️📆  👉https://t.co/tRafUaUTEe https://t.co/jjsDni3JTk 
Processed: permitimos informar programación fecha liga aguila balón fútbol índice revés apuntando derecha 
Polarity, subjectivity: 0.06507936507936506 0.4123015873015873 

Original: 🎊Felicitaciones🎊  🥳👏Felicidades a Luis Jiménez y a @anrive10 por sus cumpleaños, que no les falte la paz, la salud, el amor y que todos sus sueños se hagan realidad.👏🥳  🌟¡Enhorabuena!🌟 https://t.co/lS5CJLgDE1 
Processed: bola confeti felicitaciones bola confeti cara fiesta aplaudir felicidades luis jiménez cumpleaños falte paz salud amor sueños hagan realidad aplaudir cara fiesta enhorabuena 
Polarity, subjectivity: 0.425 0.625 

Original: Todo se cuenta en La Otra Cara del FPC🎙  🔴Hablamos con Alberto Gamero, técnico de @cdtolima y nos contó de sus títulos, de sus experiencias como jugador, como técnico y también de otras curiosidades que no te puedes perder.🔴 

...

Original: Here is a new product in my Store! Check it out! https://t.co/tjNDKvBVxf 
Processed: here is new product in my store check it out 
Polarity, subjectivity: 0.0 0.0 

Original: My new photoset is live. Login and enjoy! https://t.co/tjNDKvBVxf https://t.co/vhZBJQS1Ay 
Processed: my new photoset is live login and enjoy 
Polarity, subjectivity: 0.0 0.0 

Original: My new video is really hot! Check it out!  https://t.co/tjNDKvBVxf https://t.co/DhDM2VcyjU 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: Here is a new product in my Store! Check it out! https://t.co/tjNDKvBVxf 
Processed: here is new product in my store check it out 
Polarity, subjectivity: 0.0 0.0 

Original: My new video is really hot! Check it out!  https://t.co/tjNDKvBVxf https://t.co/7AKIdz7Xoq 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: Here is a new product in my Store! Check it out! https://t.co/tjNDKvBVxf 
Pro

Original: Dia caluroso de producciÃ³n -   https://t.co/tjNDKvBVxf https://t.co/vybfUFrq6C 
Processed: caluroso producciã³n 
Polarity, subjectivity: 0.25 0.8500000000000001 

Original: I liked a @YouTube video https://t.co/pTj5LUDuDw ¿Un revolcón porno con Andrea García o Cristian Cipriani? 
Processed: liked video revolcón porno andrea garcía cristian cipriani 
Polarity, subjectivity: 0.6 0.8 

Original: My new video is really hot! Check it out!  https://t.co/tjNDKvBVxf https://t.co/ixs6TLoJDp 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: De vuelta a la oficina.  https://t.co/tjNDKvBVxf https://t.co/E5jzgX9I0h 
Processed: vuelta oficina 
Polarity, subjectivity: 0.0 0.0 

Original: La primera novela de sexo de Instagram.: https://t.co/bHz0E4MCZP via @YouTube 
Processed: novela sexo instagram via 
Polarity, subjectivity: 0.0 0.0 

Original: Te espero en mi instagram privado https://t.co/68LQRoxXJe https://t.co/n0yWKAem8x 
Processed: esper

Original: Here is a new product in my Store! Check it out! https://t.co/OxT5JIzKwT 
Processed: here is new product in my store check it out 
Polarity, subjectivity: 0.0 0.0 

Original: My new video is really hot! Check it out!  https://t.co/OxT5JIzKwT https://t.co/kSM0ppTS23 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: My new video is really hot! Check it out!  https://t.co/OxT5JIzKwT https://t.co/rlKhbjecXK 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: Here is a new product in my Store! Check it out! https://t.co/OxT5JIzKwT 
Processed: here is new product in my store check it out 
Polarity, subjectivity: 0.0 0.0 

Original: My new video is really hot! Check it out!  https://t.co/OxT5JIzKwT https://t.co/Vu2NnE65lQ 
Processed: my new video is really hot check it out 
Polarity, subjectivity: 0.0 0.0 

Original: El mundo según Andrea Garcia: https://t.co/XyAC34uTda via @YouTube 
Processe

Original: @afro_candombera toda mi solidaridad y apoyo. Esperamos que las autoridades actúen y en cualquier caso  debemos activar todas las redes de apoyo posibles para protegernos. https://t.co/WXqwNSXXMQ 
Processed: solidaridad apoyo esperamos autoridades actúen caso debemos activar redes apoyo posibles protegernos 
Polarity, subjectivity: 0.0 1.0 

Original: Les comparto esta reflexión sobre la construcción de la paz que me dio la oportunidad de hacer la @UniversidadUNAD. Agradezco estos espacios para el diálogo que se generan desde la academia. https://t.co/GDwj3VPWya 
Processed: comparto reflexión construcción paz oportunidad agradezco espacios diálogo generan academia 
Polarity, subjectivity: 0.0 0.0 

Original: Hablemos del Presupuesto General de la Nación. Este proyecto va andando y mientras se construyen a diario las cortinas de humo, se niegan los recursos para la inversión social y la paz ¡Ojo con eso manitas y manitos! https://t.co/pgYVf4pVbD 
Processed: hablemos presupuest

IpyExit: 

In [13]:
print(len(text_analysis), len(avg_eng))
print(avg_eng[-1])
last_id = str(float('3284817049'))
text_analysis = [l for l in text_analysis if not last_id in l]
avg_eng = [l for l in avg_eng if not last_id in l]
print(len(text_analysis), len(avg_eng))

5180 57
[61645096, 287.775956284153, [('my', 132), ('new', 132), ('is', 126)], [('andreagarcia', 23), ('latigrapro', 12), ('latigra', 4)], [('andreagarcia', 23), ('latigrapro', 12), ('latigra', 4)]]
5180 57


In [14]:
# Save tweet analysis in a dataframe
analysis_results = pd.DataFrame(text_analysis[1:])
analysis_results.columns = text_analysis[0]
analysis_results["id"] = pd.to_numeric(analysis_results["id"])
analysis_results.to_csv('analysis_results.csv', index=False)
analysis_results.head()

,id,tweet_id,engagement,effective_length,polarity,subjectivity
0,426146744.0,1.142850e+18,42,125,0.600000,0.8
1,426146744.0,1.142570e+18,1441,62,0.000000,0.0
2,426146744.0,1.141490e+18,611,20,0.200000,0.4
3,426146744.0,1.141420e+18,16,184,0.366667,0.5
4,426146744.0,1.141130e+18,180,261,-0.600000,0.9


In [15]:
user_network_data=pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/user_network_data.csv')
user_network_data.head()

,id,katz,betweenness,authority
0,426146744.0,1.580982,0.000668,0.000029
1,162926902.0,3.461433,0.005760,0.000456
2,167435593.0,4.891715,0.003585,0.000369
3,269039712.0,1.375570,0.000193,0.000023
4,62561348.0,2.050953,0.000549,0.000141


In [16]:
# Create vertex data frame
vertices = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/vertices.csv', encoding='utf-8', engine='c')
vertices["id"] = pd.to_numeric(vertices["id"])
vertices = vertices.drop(['description'], axis=1)
vertices.head()

,id,screen_name,followers,friends,verified,created_at,listed,tipo
0,426146744.0,SteevenOrozco,655637,476,False,2011,842,opinion
1,162926902.0,VivaAirCol,210761,30037,False,2010,475,empresa
2,167435593.0,Dimayor,1314384,147,True,2010,1308,competencia
3,269039712.0,cecorrehumor,156975,392,False,2011,183,entretenimiento
4,62561348.0,ReddsColombia,25319,2517,True,2009,62,empresa


In [17]:
# Merge all user info

user_engagement = pd.DataFrame(avg_eng[1:])
user_engagement.columns = avg_eng[0]
user_engagement.head()

user_info = pd.merge(user_network_data, vertices, on='id')
user_info = pd.merge(user_info, user_engagement, on='id')
user_info.head()


,id,katz,betweenness,authority,screen_name,followers,friends,verified,created_at,listed,tipo,average_engagement,common_words,common_ht_words,common_hts
0,426146744.0,1.580982,0.000668,0.000029,SteevenOrozco,655637,476,False,2011,842,opinion,457.250000,"[(creyentes, 4), (bavaria, 4), (rt, 3)]","[(creyentes, 4), (bavaria, 3), (efecty, 1)]","[(Creyentes, 4), (MeUnoBavaria, 3), (EfectyElG..."
1,162926902.0,3.461433,0.005760,0.000456,VivaAirCol,210761,30037,False,2010,475,empresa,30.823529,"[(viva, 24), (air, 24), (hola, 21)]","[(viva, 3), (air, 3), (rosa, 3)]","[(VivaAirRosa, 3), (MiercolesDeMascotas, 2), (..."
2,167435593.0,4.891715,0.003585,0.000369,Dimayor,1314384,147,True,2010,1308,competencia,176.331210,"[(aguila, 111), (liga, 96), (fecha, 51)]","[(aguila, 111), (liga, 95), (femenina, 28)]","[(LigaAguila, 67), (LigaAguilaFemenina, 28), (..."
3,269039712.0,1.375570,0.000193,0.000023,cecorrehumor,156975,392,False,2011,183,entretenimiento,18.087379,"[(gracias, 33), (abrazo, 17), (amigo, 14)]","[(feliz, 6), (martes, 3), (tbt, 2)]","[(FelizMartes, 3), (felizcumpleañosbogotá, 1),..."
4,62561348.0,2.050953,0.000549,0.000141,ReddsColombia,25319,2517,True,2009,62,empresa,1.066667,"[(amarguras, 58), (perreo, 23), (comparte, 22)]","[(amarguras, 57), (menores, 1), (cerveza, 1)]","[(SinAmarguras, 57), (MenoresSinCerveza, 1)]"


In [18]:
user_info.to_csv('user_info.csv', index=False)

In [ ]:
# Merge user and tweet data
processed_data = pd.merge(user_info, analysis_results, on='id')
processed_data = processed_data.drop(['created_at'], axis=1)
processed_data.head()
len(processed_data)

In [28]:
#tweets = tweets.drop(['text','phone','hashtags','mentions'], axis=1)
processed_data = pd.merge(processed_data, tweets, on='tweet_id')
processed_data.head()
len(processed_data)

19861411

In [ ]:
# Write processed data to file
processed_data.to_csv('processed_data.csv', index=False)